In [1]:
# Import all requirments
import tensorflow as tf
# import tensorflow_datasets as tfds
import time
import  os
import io
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split

In [3]:
start_time = time.time()
## load data
# german_data_path = os.path.join(os.path.curdir,"./data/europarl-v7.de-en.de")
german_data_path= r"C:\Users\justi\Documents\RDPTranslation\initalTest\data\europarl-v7.de-en.de"
# english_data_path = os.path.join(os.path.curdir,"./data/europarl-v7.de-en.en")
english_data_path = r"C:\Users\justi\Documents\RDPTranslation\initalTest\data\europarl-v7.de-en.en"
print("Time required to load data --- %s seconds ---" % (time.time() - start_time))


Time required to load data --- 0.0 seconds ---


In [4]:
## Load data from file and into a dataset in memory
start_time = time.time()
def create_dataset(germ_path, eng_path):
  germ_lines = io.open(germ_path, encoding='UTF-8').read().strip().split('\n')
  eng_lines = io.open(eng_path, encoding='UTF-8').read().strip().split('\n')
  return germ_lines, eng_lines
german_data, english_data = create_dataset(german_data_path, english_data_path)
print("Time required to create dataset --- %s seconds ---" % (time.time() - start_time))



Time required to create dataset --- 4.76974630355835 seconds ---


In [5]:
## Tokenize german and english sequences
start_time = time.time()

en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# en_tokenizer.fit_on_texts(english_data)
# english_config = en_tokenizer.to_json()

# english_config_path = os.path.join(r"./tokenizerConfig/english_config.json")

english_config_path = r"C:\Users\justi\Documents\RDPTranslation\initalTest\tokenizerConfig\english_config.json"
## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# io.open(english_config_path,'w').write(english_config)
# print("Wrote english config to ", english_config_path)

## Comment lines out first time runnint to avoid reloading config
english_config = io.open(english_config_path, encoding='UTF-8').read()
en_tokenizer =  tf.keras.preprocessing.text.tokenizer_from_json(english_config)
print("Read english tokenizer input from ", english_config_path)

## Tokenize english data
data_en = en_tokenizer.texts_to_sequences(english_data)
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,padding='post')

ge_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# ge_tokenizer.fit_on_texts(german_data)
# german_config = ge_tokenizer.to_json()

# german_config_path = os.path.join(r"./tokenizerConfig/german_config.json")
german_config_path = r"C:\Users\justi\Documents\RDPTranslation\initalTest\tokenizerConfig\english_config.json"
## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# io.open(german_config_path,'w').write(german_config)
# print("Wrote german config to ", german_config_path)

## Comment lines out first time runnint to avoid reloading config
german_config = io.open(german_config_path, encoding='UTF-8').read()
ge_tokenizer =  tf.keras.preprocessing.text.tokenizer_from_json(german_config)
print("Read german tokenizer input from ", german_config_path)

## tokenize german data
data_ge = ge_tokenizer.texts_to_sequences(german_data)
data_ge = tf.keras.preprocessing.sequence.pad_sequences(data_ge,padding='post',maxlen=668)

print("Time required to tokenize data --- %s seconds ---" % (time.time() - start_time))


Read english tokenizer input from  C:\Users\justi\Documents\RDPTranslation\initalTest\tokenizerConfig\english_config.json
Read german tokenizer input from  C:\Users\justi\Documents\RDPTranslation\initalTest\tokenizerConfig\english_config.json
Time required to tokenize data --- 96.34556245803833 seconds ---


In [7]:
print("%d bytes" % (data_en.size * data_en.itemsize))


5130798448 bytes


In [6]:

start_time = time.time()
## slice data with an 80/20 split
X_train,  X_test, Y_train, Y_test = train_test_split(data_en,data_ge,test_size=0.4)
print("Time required to slice data --- %s seconds ---" % (time.time() - start_time))


Time required to slice data --- 4.167998790740967 seconds ---


In [7]:
print(data_en.shape)

(1920209, 668)


In [8]:
## Convert data to tensor and print shape of X_train (english training data)
Dtype = tf.float32

X_train = tf.convert_to_tensor(data_en,dtype=Dtype)
X_train = tf.reshape(X_train, [X_train.shape[0], X_train.shape[1], 1])
print(X_train.shape)
X_test = tf.convert_to_tensor(X_test,dtype=Dtype)
Y_train = tf.convert_to_tensor(data_ge,dtype=Dtype)
Y_train = tf.reshape(Y_train, [Y_train.shape[0], Y_train.shape[1], 1])
print(Y_train.shape)
Y_test = tf.convert_to_tensor(Y_test,dtype=Dtype)


(1920209, 668, 1)
(1920209, 668, 1)


In [9]:


## Define model (cnn seq2seq)

#Hyperparams            (from https://www.tensorflow.org/addons/tutorials/networks_seq2seq_nmt)
BATCH_SIZE = 64
BUFFER_SIZE = 128 #len(X_train)
# steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 256
rnn_units = 1024
dense_units = 1024
num_batches = len(X_train)/BATCH_SIZE
Dtype = tf.float32
print("set hyperparms")

set hyperparms


In [10]:

# input_vocab_size = len(en_tokenizer.word_index)+1
# output_vocab_size = len(ge_tokenizer.word_index)+ 1
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
example_X = tf.reshape(example_X,[example_X.shape[0],example_X.shape[1],1])
example_Y = tf.reshape(example_Y,[example_Y.shape[0],example_Y.shape[1],1])

# example_X.set_shape([example_X.shape[0],example_X.shape[1],1])
print(example_X.shape)
print(example_Y.shape)

(64, 668, 1)
(64, 668, 1)


In [9]:
num_batches = int(len(X_train)/BATCH_SIZE)
print(num_batches)


18001


In [11]:
def buildModel():
    model = tf.keras.Sequential(layers=[
        # tf.keras.layers.LSTM(64),
        # tf.keras.layers.LSTM(16),
        tf.keras.layers.Conv1D(668,kernel_size=1),
        tf.keras.layers.Conv1D(128,kernel_size=1),
        tf.keras.layers.Conv1D(64,kernel_size=1),
        tf.keras.layers.Conv1D(1,kernel_size=1),
        tf.keras.layers.Flatten(),
        # tf.keras.layers.Conv1D(64,kernel_size=1), #input_shape=(64,668)
        # tf.keras.layers.MaxPool1D(),
        # tf.keras.layers.Conv1D(16,kernel_size=1),
        # tf.keras.layers.Dense(16),
        # # # tf.keras.layers.MaxPool1D(),
        # #
        tf.keras.layers.Dense(16),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(668),

        # tf.keras.layers.Conv1DTranspose(1,kernel_size=1),
        # tf.keras.layers.Conv1DTranspose(668,kernel_size=1)

        ])
    return model


In [164]:
def buildModel2():
    model = tf.keras.Sequential(layers=[
        # tf.keras.layers.LSTM(64),
        # tf.keras.layers.LSTM(16),
        # tf.keras.layers.Conv1D(668,kernel_size=1),
        # tf.keras.layers.Conv1D(128,kernel_size=1),
        # tf.keras.layers.Conv1D(64,kernel_size=1),
        # tf.keras.layers.Conv1D(1,kernel_size=1),
        tf.keras.layers.Flatten(),
        # tf.keras.layers.Conv1D(64,kernel_size=1), #input_shape=(64,668)
        # tf.keras.layers.MaxPool1D(),
        # tf.keras.layers.Conv1D(16,kernel_size=1),
        # tf.keras.layers.Dense(16),
        # # # tf.keras.layers.MaxPool1D(),
        # #
        tf.keras.layers.Dense(668),
        tf.keras.layers.Dense(668),
        tf.keras.layers.Dense(668),
        tf.keras.layers.Dense(668),
        tf.keras.layers.Dense(668),
        tf.keras.layers.Dense(668),
        # tf.keras.layers.Conv1DTranspose(1,kernel_size=1),
        # tf.keras.layers.Conv1DTranspose(668,kernel_size=1)

        ])
    return model


In [15]:
start_time = time.time()
densemodel = buildModel2()
print("Time required to create model --- %s seconds ---" % (time.time() - start_time))

# seq2seq.summary()

## Train model and print results
densemodel.compile(optimizer= 'adam',loss="CategoricalCrossentropy")
# seq2seq.build(input_shape=(668,64))
# seq2seq.summary()
densemodel.fit(x=example_X,y=example_Y,batch_size=BATCH_SIZE,epochs=600)
densemodel.summary()
densemodel.save(filepath="./models/dense")

NameError: name 'buildModel2' is not defined

In [12]:
X_test = tf.reshape(X_test,[X_test.shape[0],X_test.shape[1],1])
Y_test = tf.reshape(Y_test,[Y_test.shape[0],Y_test.shape[1],1])
print(X_test.shape)
print(Y_test.shape)

(768084, 668, 1)
(768084, 668, 1)


In [14]:
start_time = time.time()
checkpoint_path = "models/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,save_freq=10)

seq2seq = buildModel()
print("Time required to create model --- %s seconds ---" % (time.time() - start_time))

# seq2seq.summary()

## Train model and print results
seq2seq.compile(optimizer='adam',loss="categorical_crossentropy")
# seq2seq.build(input_shape=(668,64))
# seq2seq.summary()
seq2seq.fit(x=example_X,y=example_Y,batch_size=BATCH_SIZE,epochs=100,
          callbacks=[cp_callback])
seq2seq.summary()
# seq2seq.save(filepath="./models/seq2seq")


Time required to create model --- 0.030010223388671875 seconds ---
Epoch 1/100
1/1 [==============================] - 0s 3ms/step - loss: nan
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: nan
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: nan
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: nan
Epoch 5/100
1/1 [==============================] - 0s 998us/step - loss: nan
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: nan
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: nan
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: nan
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: nan
Epoch 10/100

Epoch 00010: saving model to models/checkpoints\cp.ckpt
1/1 [==============================] - 0s 1ms/step - loss: nan
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: nan
Epoch 12/100
1/1 [=============================

In [15]:
print(X_test.shape)
print(Y_test.shape)

(768084, 668, 1)
(768084, 668, 1)


In [23]:
latest_weights = tf.train.latest_checkpoint(checkpoint_dir)
new_model = buildModel()
new_model.load_weights(latest_weights)

new_model.compile(optimizer='adam',loss="categorical_crossentropy")
loss, acc = new_model.evaluate(X_test, Y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


24003/24003 [==============================] - 830s 35ms/step - loss: nan


TypeError: cannot unpack non-iterable float object

In [22]:
print("X type")
print(X_test.dtype)
tf.cast(X_test,dtype=tf.int32)
print(X_test.dtype)
print("y type")
print(Y_test.dtype)
tf.cast(Y_test,dtype=tf.int32)
print(Y_test.dtype)


X type
<dtype: 'float32'>
<dtype: 'float32'>
y type
<dtype: 'float32'>
<dtype: 'float32'>


In [18]:
print(X_train.dtype)

<dtype: 'float32'>
